# Boosting 算法选股

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind

# 基础函数
import utilsJ_ML

d:\Anaconda\envs\py38\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# 参数设置：
https://uqer.datayes.com/v3/community/share/58f81f615296790056b7d345
http://pg.jrj.com.cn/acc/Res/CN_RES/INVEST/2016/5/31/ed36ae43-0f6e-4051-bb9c-2e9a67632d74.pdf

行情因子：
MA10, MA20, MACD, RSI

基本面因子：


In [19]:
class ml(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Keep references to lines in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Keep references to executed order & price
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Add indicators

        self.bollinger = btind.BollingerBands(period=20, devfactor=2)
        self.macdhisto = btind.MACDHisto()
        self.wr = btind.WilliamsR(period=14)

        self.updown = bt.If(self.dataclose(0) > self.dataclose(-1), 1, 0)



    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.datas[0].testadd[0])

    
    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

In [20]:
class PandasData_ML(bt.feeds.PandasData):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    lines = ('EPS','NetProfit','ROE', )

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
        ('EPS', -1),
        ('NetProfit', -1),
        ('ROE', -1),
    )

In [2]:
s_date = datetime.datetime.now()- datetime.timedelta(days=385)
e_date = datetime.datetime.now()
stock_code = '600036.SH'
utilsJ_ML.stock_finance('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',stock_code, s_date, e_date)

    trade_date     open     high      low    close      volume  openinterest
0   2021-09-22  46.6517  47.8976  46.0959  47.3513  1122596.22           0.0
1   2021-09-23  47.4376  48.4247  46.5847  46.8051   861024.53           0.0
2   2021-09-24  46.4313  47.2747  46.3834  46.7380   532039.33           0.0
3   2021-09-27  46.2780  47.7251  46.2013  47.0447   667905.15           0.0
4   2021-09-28  47.0542  48.8559  47.0447  48.3192   709925.47           0.0
..         ...      ...      ...      ...      ...         ...           ...
247 2022-09-28  33.7900  34.0000  33.4200  33.6700   487298.60           0.0
248 2022-09-29  33.9900  34.3200  33.0000  33.1500   617859.02           0.0
249 2022-09-30  33.2600  33.9500  33.1200  33.6500   539717.07           0.0
250 2022-10-10  32.9900  33.0900  31.2500  31.8000  1277991.32           0.0
251 2022-10-11  31.7900  32.0700  31.3100  31.4000   618664.47           0.0

[252 rows x 7 columns]


,trade_date,open,high,low,close,volume,openinterest,eps,netprofit_margin,roe
0,2021-09-22,46.6517,47.8976,46.0959,47.3513,1122596.22,0.0,3.62,37.5307,12.3771
1,2021-09-23,47.4376,48.4247,46.5847,46.8051,861024.53,0.0,3.62,37.5307,12.3771
2,2021-09-24,46.4313,47.2747,46.3834,46.7380,532039.33,0.0,3.62,37.5307,12.3771
3,2021-09-27,46.2780,47.7251,46.2013,47.0447,667905.15,0.0,3.62,37.5307,12.3771
4,2021-09-28,47.0542,48.8559,47.0447,48.3192,709925.47,0.0,3.62,37.5307,12.3771
...,...,...,...,...,...,...,...,...,...,...
250,2022-09-28,33.7900,34.0000,33.4200,33.6700,487298.60,0.0,2.67,39.0874,7.9484
251,2022-09-29,33.9900,34.3200,33.0000,33.1500,617859.02,0.0,2.67,39.0874,7.9484
252,2022-09-30,33.2600,33.9500,33.1200,33.6500,539717.07,0.0,2.67,39.0874,7.9484
253,2022-10-10,32.9900,33.0900,31.2500,31.8000,1277991.32,0.0,2.67,39.0874,7.9484


In [21]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(ml, printlog=True) 

    s_date = datetime.datetime.now()- datetime.timedelta(days=385)
    e_date = datetime.datetime.now()

    # Create stock Data Feed
    stock_index = '000001.SH'
    df = utilsJ_ML.index_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                        stock_index, s_date, e_date)
    #df = pd.read_csv('C:\\Users\\a\\Desktop\\Backtrader\\Data\\2019\\' + stock_index + '.csv', 
    #                converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    df['testadd'] = 1
    data = PandasData_ML(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 20000.00
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Final Portfolio Value: 20000.00
Net Profit: 0.00%
